In [ ]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))

# Results loader object does everything
from ionpy.analysis import ResultsLoader
from pathlib import Path
root = Path("/storage/vbutoi/scratch/ESE")
rs = ResultsLoader()

# For using code without restarting.
%load_ext autoreload
%autoreload 
# For using yaml configs.
%load_ext yamlmagic

In [ ]:
%%yaml results_cfg 

# EXPERIMENT SETS:
# - Calibration (no weighting): 01_18_24_All_Metrics
# - Only Foreground Loss: 01_21_24_Foreground_Calibrators
# - Balanced Loss: 01_21_24_Balanced_CE_Calibrators

log:
    root: /storage/vbutoi/scratch/ESE/inference
    load_pixel_meters: True 
    remove_shared_columns: True
    drop_nan_metric_rows: False
    min_fg_pixels: 100
    inference_paths:
        - "01_21_24_Foreground_Calibrators/WMH_Individual_Uncalibrated"
        - "01_21_24_Foreground_Calibrators/WMH_Individual_TempScaling"
        - "01_21_24_Foreground_Calibrators/WMH_Individual_VectorScaling"
        - "01_21_24_Foreground_Calibrators/WMH_Individual_DirichletScaling"
        - "01_21_24_Foreground_Calibrators/WMH_Individual_LTS"
        - "01_21_24_Foreground_Calibrators/WMH_Individual_NectarScaling"
        - "01_21_24_Foreground_Calibrators/WMH_Ensemble_Uncalibrated"
        - "01_21_24_Foreground_Calibrators/WMH_Ensemble_TempScaling"
        - "01_21_24_Foreground_Calibrators/WMH_Ensemble_VectorScaling"
        - "01_21_24_Foreground_Calibrators/WMH_Ensemble_DirichletScaling"
        - "01_21_24_Foreground_Calibrators/WMH_Ensemble_LTS"
        - "01_21_24_Foreground_Calibrators/WMH_Ensemble_NectarScaling"
    
calibration:
    conf_interval:
        - 0.5
        - 1.
    num_bins: 10
    square_diff: False 
    neighborhood_width: 3

# cal_metrics:
#     - ECE:
#         _fn: ese.experiment.metrics.ece.ece_loss
#     - CW_ECE:
#         _fn: ese.experiment.metrics.ece.cw_ece_loss
#     - Edge_ECE:
#         _fn: ese.experiment.metrics.ece.edge_ece_loss
#     - ELM:
#         _fn: ese.experiment.metrics.elm.elm_loss
#     - Foreground_ECE:
#         _fn: ese.experiment.metrics.ece.ece_loss
#         ignore_index: 0
#     - Foreground_CW_ECE:
#         _fn: ese.experiment.metrics.ece.cw_ece_loss
#         ignore_index: 0
#     - Foreground_Edge_ECE:
#         _fn: ese.experiment.metrics.ece.edge_ece_loss
#         ignore_index: 0       
#     - Foreground_ELM:
#         _fn: ese.experiment.metrics.elm.elm_loss
#         ignore_index: 0

In [ ]:
from ese.experiment.analysis.inference import load_cal_inference_stats

image_info_df = load_cal_inference_stats(
    results_cfg=results_cfg
)

## Let's try to see if there is any hope with having better ECE/ELM makes better ensembles. Note that this isn't a conclusive result just because the number of samples per images that are used to calculate ECE/ELM are not sufficient to get actual statistical quantities.

### We have to add to each row a column that is the difference betweeen the row's metric_score and the metric_score corresponding to the same image metric as mean uncalibrated UNet performance. 

In [ ]:
# Step 1: Get the rows corresponding to a unet aveages across multiple seeds with no calibration.
average_unet_row = image_info_df[(image_info_df['pretrained_seed'] == 'Average') & (image_info_df['calibrator'] == 'Uncalibrated')]

In [ ]:
unique_datapoint_cols = ['data_id', 'slice_idx', 'image_metric', 'groupavg_image_metric']

In [ ]:
# Sanity check that for each datapoint we only have one average unet row.
num_avg_unets_per_datapoint = average_unet_row.groupby(unique_datapoint_cols).size()
assert num_avg_unets_per_datapoint.max() == 1,\
    f"There should be only one row for each data_id, slice_idx, image_metric, and groupavg image metric combination, got {num_avg_unets_per_datapoint}."

In [ ]:
# Make sure there are no NaNs in the average UNet rows.
assert average_unet_row['metric_score'].isna().sum() == 0, "There should be no NaNs in metric_score of UNet rows."
assert average_unet_row['groupavg_metric_score'].isna().sum() == 0, "There should be no NaNs in groupavg_metric_score of UNet rows."

In [ ]:
# Step 2: Merge based on 'image_metric', 'subject_id', and 'slice_idx'
info_df_w_avg_unet_cols = pd.merge(
    image_info_df, 
    average_unet_row[unique_datapoint_cols + ['metric_score', 'groupavg_metric_score']], 
    on=unique_datapoint_cols, 
    how='left', 
    suffixes=('', '_average_unet')
    )

In [ ]:
# Step 3: Calculate the difference
info_df_w_avg_unet_cols['metric_delta'] = info_df_w_avg_unet_cols['metric_score'] - info_df_w_avg_unet_cols['metric_score_average_unet'] # Current - Baseline
info_df_w_avg_unet_cols['groupavg_metric_delta'] = info_df_w_avg_unet_cols['groupavg_metric_score'] - info_df_w_avg_unet_cols['groupavg_metric_score_average_unet'] # Current - Baseline
# Drop those columns
info_df_w_delta = info_df_w_avg_unet_cols.drop(columns=['metric_score_average_unet', 'groupavg_metric_score_average_unet'])

In [ ]:
# info_df_w_delta = info_df_w_delta.dropna(subset=['metric_delta', 'groupavg_metric_delta'])

# We want to drop all of the rows of (data_id, slice_idx) that have NaNs in the delta columns.
# This means that the average unet row was not present for that (data_id, slice_idx) combination.

# Get the rows that have NaNs in the delta columns
# nan_delta_rows = info_df_w_delta[info_df_w_delta['metric_delta'].isna() | info_df_w_delta['groupavg_metric_delta'].isna()]
# # Drop the nan delta rows
# info_df_w_delta = info_df_w_delta.drop(nan_delta_rows.index)

## Now we can look at trends! We want to make some scatterplots to look at relationships between calibration scores and their relative improvement over the baseline.   

In [ ]:
# First, we want only the rows corresponding to group metrics, no longer looking at seeds.
grouped_models_df = info_df_w_delta[info_df_w_delta['model_type'] == 'group'].reset_index(drop=True)

In [ ]:
# SANITY CHECKS, MAKE SURE THAT FOR CALIBRATORS UNCALIBRATED, TEMPERATURE_SCALING, LTS
for calibrator in ["Uncalibrated", "Temperature_Scaling", "LTS"]:
    unique_qual_metrics = grouped_models_df[grouped_models_df['metric_type'] == 'quality']['image_metric'].unique()
    for quality_metric in unique_qual_metrics:
        # Checkign that the delta is 0 for the calibrator and the quality_metric
        rows = grouped_models_df[
            (grouped_models_df['calibrator'] == calibrator) & 
            (grouped_models_df['image_metric'] == quality_metric) &
            (grouped_models_df['method_name'] == 'UNet (seed=Average)')
        ]
        assert (rows['metric_delta'] == 0).all(),\
            f"Delta from base should be 0 for the calibrator {calibrator} and the quality metric {quality_metric}, got {rows['metric_delta']}."

In [ ]:
# Make a bunch of new rows where the image_metric is the groupavg_image_metric and the metric_score is the groupavg_metric_score and the metric_delta is the groupavg_metric_delta
groupavg_rows = grouped_models_df.copy()
groupavg_rows['image_metric'] = groupavg_rows['groupavg_image_metric']
groupavg_rows['metric_score'] = groupavg_rows['groupavg_metric_score']
groupavg_rows['metric_delta'] = groupavg_rows['groupavg_metric_delta']
# Drop the groupavg columns
standard_image_rows = grouped_models_df.drop(columns=['groupavg_image_metric', 'groupavg_metric_score', 'groupavg_metric_delta']) 
groupavg_rows = groupavg_rows.drop(columns=['groupavg_image_metric', 'groupavg_metric_score', 'groupavg_metric_delta'])
# Concatenate the two
grouped_models_df = pd.concat([standard_image_rows, groupavg_rows], axis=0, ignore_index=True)

In [ ]:
# Create a pivot table with 'metric_type' as columns
pivot_grouped_models_df = grouped_models_df.pivot_table(
    index=['configuration', 'method_name', 'calibrator', 'data_id', 'slice_idx'],
    values=['metric_score', 'metric_delta'], 
    columns=['metric_type', 'image_metric'], 
    aggfunc='mean'
).reset_index()

In [ ]:
# Make a copy so that we can modify the column names
pivot_perf_per_datpoint = pivot_grouped_models_df.copy()
# Make new column names.
new_cols = []
for col in pivot_grouped_models_df.columns.values:
    if col[0] == 'metric_delta':
        new_cols.append(f'delta_{col[-1]}')
    elif col[-1] == '':
        new_cols.append(col[0])
    else:
        new_cols.append(col[-1])
# Set the column names to be the lowest non empty level per column in the multi-index
pivot_perf_per_datpoint.columns = new_cols

In [ ]:
pivot_perf_per_datpoint.columns

In [ ]:
# We want to sort the pivot_df so that the order of the method names is
# UNet (seed=Average), Ensemble (mean, logits), Ensemble (mean, probs),
# and the order of the calibrators is Uncalibrated, Temperature Scaling, LTS, Vector Scaling, Dirichlet Scaling
method_order_name = ['UNet (seed=Average)', 'Ensemble (mean, logits)', 'Ensemble (mean, probs)']
calibrator_order_name = ['Uncalibrated', 'Temperature_Scaling', 'LTS', 'NECTAR_Scaling', 'Vector_Scaling', 'Dirichlet_Scaling']
# Sort the methods
pivot_perf_per_datpoint['method_name'] = pivot_perf_per_datpoint['method_name'].astype('category')
pivot_perf_per_datpoint['method_name'] = pivot_perf_per_datpoint['method_name'].cat.reorder_categories(method_order_name)
# Sort the calibrators
pivot_perf_per_datpoint['calibrator'] = pivot_perf_per_datpoint['calibrator'].astype('category')
pivot_perf_per_datpoint['calibrator'] = pivot_perf_per_datpoint['calibrator'].cat.reorder_categories(calibrator_order_name)
# Sort the dataframe
sorted_pivot_perf_per_datpoint = pivot_perf_per_datpoint.sort_values(by=['method_name', 'calibrator'])

## Looking at change in *predicted ensemble* calibration vs change in Dice.

In [ ]:
# Assert that each subplot will have the same number of datapoints on it.
num_per_config = sorted_pivot_perf_per_datpoint.groupby(['method_name', 'calibrator']).size()
assert len(num_per_config.unique()) == 1,\
    f"Each subplot should have the same number of datapoints on it, got {num_per_config}."

In [ ]:
num_per_config

In [ ]:
from ese.experiment.analysis.plot_utils import plot_method_vs_calibrator_scatterplots

In [ ]:
for foreground_option in ['', 'Foreground-']:
    for group_option in ['', 'GroupAvg_']:
        x_metric_name = f'delta_{group_option}Image_{foreground_option}Edge-ECE'
        y_metric_name = f'delta_Dice'
        # Plot the relationship between the two metrics
        plot_method_vs_calibrator_scatterplots(
            df=sorted_pivot_perf_per_datpoint, 
            x=x_metric_name, 
            y=y_metric_name,
            sharex=False,
            sharey=False,
            height=5
        )
        break
    break

In [ ]:
for foreground_option in ['', 'Foreground-']:
    for group_option in ['', 'GroupAvg_']:
        x_metric_name = f'delta_{group_option}Image_{foreground_option}Edge-ECE'
        y_metric_name = f'delta_Dice'
        # Bin the predictions
        num_bins = 100
        # Make a copy of the dataframe
        pivot_df_copy = sorted_pivot_perf_per_datpoint.copy()
        # Use pandas qcut to create quantile-based bins and calculate average x and y values within each bin
        pivot_df_copy['bin'] = pd.qcut(pivot_df_copy[x_metric_name].rank(method='first'), q=num_bins, labels=False)
        binned_pivot_per_datapoint = pivot_df_copy.groupby(['bin', 'method_name', 'calibrator']).agg({
            x_metric_name: 'mean', 
            y_metric_name: 'mean'
            }).reset_index()
        # Assert that the number of points in the plot is the same
        num_per_config = binned_pivot_per_datapoint.groupby(['method_name', 'calibrator']).size()
        assert num_per_config.max() == num_per_config.min(),\
            f"Each subplot should have the same number of datapoints on it, got {num_per_config}."
        # Plot the relationship between the two metrics
        plot_method_vs_calibrator_scatterplots(
            df=binned_pivot_per_datapoint, 
            x=x_metric_name, 
            y=y_metric_name,
            sharex=False,
            sharey=False,
            height=5
        )
        break
    break